# ML in Finance Group Project
### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch
## 1. Preperation of Datasets
In this part of the code the data is imported from the Wharton data source. It's then cleaned up and put into usable attribute matrices for further feature selection.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

### Market Data from Wharton
The financial data from the Wharton Database is loaded in. The redundant identification columns and columns which are more than 2/3 empty values are dropped.  

In [2]:
# Load in data from the Wharton Database
wharton = pd.read_csv('Data/Whartondata.csv', sep=',', header=0)

# Delete unusable columns
wharton = wharton.drop(['NAMEENDT','SHRCD','EXCHCD','SICCD','TICKER','COMNAM', 'NCUSIP', 'TSYMBOL',
              'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD'], 1)
wharton = wharton.drop(['DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'HSICMG', 'HSICIG', 'DIVAMT',
              'SHRCLS', 'ACPERM', 'ACCOMP', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'NMSIND',
              'MMCNT', 'NSDINX', 'DCLRDT', 'PAYDT', 'RCRDDT', 'DISTCD', 'FACPR', 'FACSHR',
              'TRTSCD', ], axis=1)

# Rename PERMNO column
wharton.columns.values[0] = 'permno'

# Format date into string with only year and month
wharton['date'] = wharton.date.astype(str).str[:4] + '-' + wharton.date.astype(str).str[4:6]


# Calculate SPREAD manually
wharton['SPREAD'] = wharton['BID'] - wharton['ASK']

# print(wharton.isnull().sum())
# display(wharton.head())

### Financial Ratios from Wharton
The financial ratios from the Wharton Database (received on OLAT) are loaded in. The adate and qdate are dropped because they are not relevant. Formatting "divyeld" from percentage to float.

In [3]:
# Load in the financial ratios from OLAT
ratios = pd.read_csv('Data/Ratios.csv', sep=',', header=0)

# Delete unusable columns
ratios = ratios.drop(['adate', 'qdate'], axis=1)

ratios.columns.values[1] = 'date'
ratios['date'] = ratios.date.str[6:] + '-' + ratios.date.str[3:5]

# Remove percentages in row "divyield" and divide with 100 (so its decimal percentage) with string split
ratios['divyield'] = ratios['divyield'].str.rstrip('%').astype('float')/100

# print(ratios.isnull().sum())
# display(ratios.head())

### Merging the Dataset

In [4]:
# Merging the two dataframes on date and permno
data = pd.merge(wharton, ratios, left_on=['date', 'permno'], right_on=['date', 'permno'])
display(data.head())

,permno,date,NAICS,PRIMEXCH,TRDSTAT,SECSTAT,CUSIP,SHRFLG,SHRENDDT,BIDLO,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,10107,2006-01,511210.0,Q,A,R,59491810,0.0,20060330.0,26.28,...,1.296,0.151,0.025,0.0,0.055,6.019,11.538,0.0114,15.506,2.109
1,10107,2006-02,511210.0,Q,A,R,59491810,0.0,NaN,26.39,...,1.323,0.151,0.025,0.0,0.036,6.281,10.280,0.0134,14.555,1.838
2,10107,2006-03,511210.0,Q,A,R,59491810,0.0,20060423.0,26.85,...,1.323,0.151,0.025,0.0,0.036,6.293,10.410,0.0132,14.739,1.842
3,10107,2006-04,511210.0,Q,A,R,59491810,0.0,20060629.0,24.15,...,1.323,0.151,0.025,0.0,0.036,5.573,9.239,0.0149,13.081,1.666
4,10107,2006-05,511210.0,Q,A,R,59491810,0.0,NaN,22.56,...,1.388,0.150,0.025,0.0,0.024,5.496,0.709,0.0159,-5.842,1.480


### Calculating Future Returns and Creating Response Vectors

In [5]:
# Creating a future returns for different forcasting periods
forcast_periods = [1, 3, 6, 12]
for i in forcast_periods:
    data['return_' + str(i)] = np.where((data['permno'] == data['permno'].shift(i)),
                                        (data['PRC'] / data['PRC'].shift(i)).astype(np.float64), np.nan)

# Create the response vectors
data_1 = data.dropna(subset=['return_1'])
response_1 = pd.DataFrame(np.where(data_1.return_1 >= 1, 1, 0))

# Export the response vectors
response_1.to_csv('Data/generated_datasets/response_1.csv', index=False)

### Creating Attribute Matrices and a Response Vectors using only Ratios

In [6]:
# Creating the attribute matrices
attributes_ratios_1 = data_1.iloc[:, 28:-4]

# Export the attribute matrices
attributes_ratios_1.to_csv('Data/generated_datasets/attributes_ratios_1.csv', index=False)

### Adding additonal Features

In [7]:
# Adding reporting period as seasonality attribute
data['SEASON'] = data['date'].str[5:].astype(str)

# Breakdown of categorical data
data['NAICS'] = data['NAICS'].astype(str)
data['RET'] = data['RET'].astype(np.float64)
data['RETX'] = data['RETX'].astype(np.float64)

data = pd.get_dummies(data, columns=['SEASON', 'NAICS', 'PRIMEXCH', 'TRDSTAT', 'SECSTAT', 'CUSIP', ])
display(data.head())

,permno,date,SHRFLG,SHRENDDT,BIDLO,ASKHI,PRC,VOL,RET,BID,...,CUSIP_65410610,CUSIP_71708110,CUSIP_74271810,CUSIP_88579Y10,CUSIP_89417E10,CUSIP_91301710,CUSIP_91324P10,CUSIP_92343V10,CUSIP_92826C83,CUSIP_93114210
0,10107,2006-01,0.0,20060330.0,26.28,28.15,28.15,13948013.0,0.076482,28.15,...,0,0,0,0,0,0,0,0,0,0
1,10107,2006-02,0.0,NaN,26.39,28.04,26.87,11088149.0,-0.042273,26.87,...,0,0,0,0,0,0,0,0,0,0
2,10107,2006-03,0.0,20060423.0,26.85,27.89,27.21,14514337.0,0.012653,27.24,...,0,0,0,0,0,0,0,0,0,0
3,10107,2006-04,0.0,20060629.0,24.15,27.74,24.15,14689919.0,-0.112459,24.16,...,0,0,0,0,0,0,0,0,0,0
4,10107,2006-05,0.0,NaN,22.56,24.29,22.65,23651189.0,-0.058385,22.70,...,0,0,0,0,0,0,0,0,0,0


### Creating Attribute Matrices and a Response Vectors using additional attributes

In [8]:
data_1 = data.dropna(subset=['return_1'])

# Creating the attribute matrices
attributes_additional_1 = data_1.drop(['permno', 'date', 'return_1', 'return_3', 'return_6', 'return_12'], axis=1)

# Export the attribute matrices
attributes_additional_1.to_csv('Data/generated_datasets/attributes_additional_1.csv', index=False)